In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential

In [2]:
td = TensorDict(x=torch.randn(5, 3), z=torch.randn(5, 8), batch_size=[5])

linear0 = TensorDictModule(
    nn.Linear(3, 128), in_keys=[("input", "x")], out_keys=[("hidden", "linear0")]
)
relu0 = TensorDictModule(
    torch.relu, in_keys=[("hidden", "linear0")], out_keys=[("hidden", "relu0")]
)
linear1 = TensorDictModule(
    nn.Linear(128, 128), in_keys=[("hidden", "relu0")], out_keys=[("hidden", "linear1")]
)
relu1 = TensorDictModule(nn.ReLU(), in_keys=[("hidden", "linear1")], out_keys=[("hidden", "relu1")])
linear2 = TensorDictModule(
    nn.Linear(128, 3), in_keys=[("hidden", "relu1")], out_keys=[("hidden", "linear2")]
)

In [3]:
TensorDictSequential(linear0, relu0)

TensorDictSequential(
    module=ModuleList(
      (0): TensorDictModule(
          module=Linear(in_features=3, out_features=128, bias=True),
          device=cpu,
          in_keys=[('input', 'x')],
          out_keys=[('hidden', 'linear0')])
      (1): TensorDictModule(
          module=<built-in method relu of type object at 0x130e20088>,
          device=cpu,
          in_keys=[('hidden', 'linear0')],
          out_keys=[('hidden', 'relu0')])
    ),
    device=cpu,
    in_keys=[('input', 'x')],
    out_keys=[('hidden', 'linear0'), ('hidden', 'relu0')])

In [11]:
block = nn.Sequential(OrderedDict(linear=nn.Linear(10, 10)))
block['linear']

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
for key in list(td.keys()):
    td.rename_key_(key, f"input.{key}")
td

TensorDict(
    fields={
        input.x: Tensor(shape=torch.Size([5, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        input.z: Tensor(shape=torch.Size([5, 8]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([5]),
    device=None,
    is_shared=False)

In [36]:
td[("input", "z")] = torch.randn(5, 16)
td

TensorDict(
    fields={
        input.x: Tensor(shape=torch.Size([5, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        input.z: Tensor(shape=torch.Size([5, 8]), device=cpu, dtype=torch.float32, is_shared=False),
        input: TensorDict(
            fields={
                z: Tensor(shape=torch.Size([5, 16]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([5]),
            device=None,
            is_shared=False)},
    batch_size=torch.Size([5]),
    device=None,
    is_shared=False)

In [43]:
[(key, val.shape) for key, val in td["input"].items(True, True)]

[('z', torch.Size([5, 16]))]

In [14]:
from torch.optim import Adam

In [20]:
W = nn.Linear(10, 10)

opt = Adam(W.parameters())

type(opt)(W.parameters())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [9]:
import numpy as np
import pandas as pd

N = 1024
df = pd.DataFrame(dict(x=np.random.rand(N), y=np.random.rand(N), z=np.random.rand(N)))

df

,x,y,z
0,0.296124,0.468575,0.366797
1,0.663179,0.320301,0.994041
2,0.782132,0.870233,0.104931
3,0.843557,0.683359,0.988978
4,0.161203,0.232490,0.329125
...,...,...,...
1019,0.716826,0.380671,0.951075
1020,0.571119,0.096809,0.016450
1021,0.254935,0.618807,0.610960
1022,0.648206,0.728535,0.789112


In [35]:
df[slice(None)]

,x,y,z
0,0.296124,0.468575,0.366797
1,0.663179,0.320301,0.994041
2,0.782132,0.870233,0.104931
3,0.843557,0.683359,0.988978
4,0.161203,0.232490,0.329125
...,...,...,...
1019,0.716826,0.380671,0.951075
1020,0.571119,0.096809,0.016450
1021,0.254935,0.618807,0.610960
1022,0.648206,0.728535,0.789112


In [24]:
index = np.random.randint(N, size=(128,))

# %timeit [df.iloc[i] for i in index]

590 μs ± 3.22 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
import torch

A = np.random.randn(128, 4)

%timeit torch.stack([torch.as_tensor(a) for a in A])
%timeit torch.as_tensor(A)

107 μs ± 1.1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
685 ns ± 7.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [29]:
%timeit torch.as_tensor(list(A))

76.9 μs ± 1.27 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [37]:
_ = """
[
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "targets": [0.5, 2.0]}
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "targets": [0.7, 2.5]}
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "targets": [0.2, 3.0]}
]

< OR >

[
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "pki_a": 2.0, "pki_b: 3.0}
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "pki_a": 2.0, "pki_b: 3.0}
    {"G": Graph(...), "fp": [1, 0, ..., 1, 0, 0], "pki_a": 2.0, "pki_b: 3.0}
]

-->

TensorDict{
    "input.G": BatchedGraph(...),
    "input.fp": Tensor(...),
    "input.targets": Tensor(...),
}
"""

In [ ]:
# df["targets"] = df[['x', 'y']].tolist()
df

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [46]:
df['targets'] = df[['x', 'y']].values.tolist()
df

,x,y,z,targets
0,0.296124,0.468575,0.366797,"[0.29612381919886, 0.46857461860676564]"
1,0.663179,0.320301,0.994041,"[0.6631789198415855, 0.32030146296918693]"
2,0.782132,0.870233,0.104931,"[0.7821318127290143, 0.8702330047875827]"
3,0.843557,0.683359,0.988978,"[0.843556561935685, 0.6833593333816355]"
4,0.161203,0.232490,0.329125,"[0.1612031653035363, 0.23248993697287146]"
...,...,...,...,...
1019,0.716826,0.380671,0.951075,"[0.7168261436777018, 0.3806711249861484]"
1020,0.571119,0.096809,0.016450,"[0.5711189832137389, 0.09680853516491938]"
1021,0.254935,0.618807,0.610960,"[0.25493538220931045, 0.6188066910290669]"
1022,0.648206,0.728535,0.789112,"[0.6482056347418593, 0.7285350929461654]"


In [48]:
df.iloc[0].targets

[0.29612381919886, 0.46857461860676564]